In [115]:
'''
This notebook is oriented to use a given dataset customized to contains 6 features and training a classification tree
This features are: cqt, rmse, energy, mfccs, chromagram and spectral contrast
This notebook is currently in development phase, so contains a considerably quantity of test code

'''

'\nThis notebook is oriented to use a given dataset customized to contains 6 features and training a classification tree\nThis features are: cqt, rmse, energy, mfccs, chromagram and spectral contrast\nThis notebook is currently in development phase, so contains a considerably quantity of test code\n\n'

In [8]:
import tensorflow as tf
import pandas as pd
from pathlib import Path
import librosa, librosa.display
import matplotlib.pyplot as plt
import numpy as np
import gc

# Reading Audio Data


# features = pd.read_csv('Datasets/2000Spoti.csv', encoding='latin-1', thousands=',')

features = pd.read_csv('Dataframes/var_mean_dict.csv', encoding='latin-1', thousands=',') # Media genera 0.62-0.67 con todas las caracteristicas **best result2**

print(features.head(5))

print('done')

   Unnamed: 0  Genre  Energy_Mean  RMSE_Mean  ZCR_Mean  CQT_Mean  \
0           0  Blues    40.490492   0.130192     55050  0.001573   
1           1  Blues    23.686239   0.095925     37156  0.003940   
2           2  Blues    68.631996   0.175582     50560  0.007814   
3           3  Blues    53.671735   0.141053     22060  0.003624   
4           4  Blues    21.844622   0.091499     67244  0.001955   

   Spectral_Contrast_Mean  Chromagram_Mean  Tempo_Mean  STFT_Mean  ...  \
0               20.526699         0.278385  123.046875   0.000037  ...   
1               20.676128         0.266356  107.666016   0.000021  ...   
2               22.197517         0.272697  161.499023   0.000009  ...   
3               21.426268         0.252442  172.265625   0.000005  ...   
4               21.466338         0.269720  135.999178   0.000014  ...   

   MFCCS3  MFCCS4  MFCCS5  MFCCS6  MFCCS7  MFCCS8  MFCCS9  MFCCS10  MFCCS11  \
0     226     125     114      82      44      39      31       57 

In [9]:

#extracting labels and features

labelColumn = 'Genre'

labels = np.array(features[labelColumn])
print(np.unique(labels))

print("The labels of out dataset are:", np.unique(labels))
print("And their shape is:", labels.shape)


# Only var no CQT
# featureColumsToDrop = ['Unnamed: 0','STFT_Mean.1', 'CQT_Var', 'STFT_Var.1', 'Genre', 'Energy_Mean', 'RMSE_Mean', 'ZCR_Mean', 'CQT_Mean', 'Spectral_Contrast_Mean', 'Chromagram_Mean', 'Tempo_Mean', 'STFT_Mean', 
#                       'Autocorrelation_Mean', 'STFT_Mean', 'Spectral_Centroid_Mean', 'Rolloff_Mean']

# Only var with CQT
# featureColumsToDrop = ['Unnamed: 0', 'Energy_Mean', 'RMSE_Mean', 'ZCR_Mean', 'CQT_Mean', 'Spectral_Contrast_Mean', 'Chromagram_Mean', 'Tempo_Mean', 'STFT_Mean', 
#                       'Autocorrelation_Mean', 'STFT_Mean', 'Spectral_Centroid_Mean', 'Rolloff_Mean','STFT_Mean.1', 'STFT_Var.1', 'Genre']

# Only mean no CQT

# featureColumsToDrop = ['Unnamed: 0','Energy_Var', 'RMSE_Var', 'ZCR_Var', 'CQT_Var', 'Spectral_Contrast_Var', 'Chromagram_Var', 'Tempo_Var', 'STFT_Var',
#                       'Autocorrelation_Var', 'STFT_Var', 'Spectral_Centroid_Var', 'Rolloff_Var', 'CQT_Mean', 'STFT_Mean.1', 'STFT_Var.1', 'Genre']

# Only mean with CQT
# featureColumsToDrop = ['Unnamed: 0', 'Energy_Var', 'RMSE_Var', 'ZCR_Var', 'CQT_Var', 'Spectral_Contrast_Var', 'Chromagram_Var', 'Tempo_Var', 'STFT_Var',
#                       'Autocorrelation_Var', 'STFT_Var', 'Spectral_Centroid_Var', 'Rolloff_Var',  'STFT_Mean.1', 'STFT_Var.1', 'Genre']


# Mean and var with CQT

featureColumsToDrop = ['Unnamed: 0', 'STFT_Mean.1', 'STFT_Var.1', 'Genre']


# NO MFCCS mean and var

# featureColumsToDrop = ['Unnamed: 0', 'STFT_Mean.1', 'STFT_Var.1', 'Genre',
#                        'Energy_Mean', 'RMSE_Mean', 'ZCR_Mean', 'CQT_Mean', 'Spectral_Contrast_Mean', 'Chromagram_Mean', 'Tempo_Mean', 'STFT_Mean', 
#                       'Autocorrelation_Mean', 'STFT_Mean', 'Spectral_Centroid_Mean', 'Rolloff_Mean',
#                        'MFCCS0','MFCCS1','MFCCS2','MFCCS3','MFCCS4','MFCCS5','MFCCS6','MFCCS7','MFCCS8','MFCCS9','MFCCS10', 'CQT_Mean','MFCCS11','MFCCS12', 'CQT_Var', 'STFT_Var.1', 'Genre']  

features = features.drop(featureColumsToDrop, 1)

print("The features of out dataset are:")

features.head(5)

['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock']
The labels of out dataset are: ['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock']
And their shape is: (1000,)
The features of out dataset are:


,Energy_Mean,RMSE_Mean,ZCR_Mean,CQT_Mean,Spectral_Contrast_Mean,Chromagram_Mean,Tempo_Mean,STFT_Mean,Autocorrelation_Mean,Spectral_Centroid_Mean,...,MFCCS3,MFCCS4,MFCCS5,MFCCS6,MFCCS7,MFCCS8,MFCCS9,MFCCS10,MFCCS11,MFCCS12
0,40.490492,0.130192,55050,0.001573,20.526699,0.278385,123.046875,0.000037,590.450073,1784.165850,...,226,125,114,82,44,39,31,57,60,35
1,23.686239,0.095925,37156,0.003940,20.676128,0.266356,107.666016,0.000021,1313.715454,1530.176679,...,774,341,139,94,82,75,64,83,81,55
2,68.631996,0.175582,50560,0.007814,22.197517,0.272697,161.499023,0.000009,577.048035,1552.811865,...,197,252,85,80,59,34,39,40,40,51
3,53.671735,0.141053,22060,0.003624,21.426268,0.252442,172.265625,0.000005,5695.121582,1070.106615,...,327,183,108,50,55,36,82,48,40,35
4,21.844622,0.091499,67244,0.001955,21.466338,0.269720,135.999178,0.000014,89.149872,1835.004266,...,187,150,54,76,52,26,107,66,49,37


In [15]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split

def train_clasify():
    """
    Method that train a RF Classifier and return a metrics report with
        - Recall by label
        - Precision by label
        - F1-score by label
        - Avg weighted recall
        - Avg weighted precision
        - Avg weighted f1-score
        - Avg macro recall
        - Avg macro recall
        - Avg macro recall
        - Avg total accuracy
    """

    # Split the data into training and testing sets
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25)

    print('Training Features Shape:', train_features.shape)
    print('Training Labels Shape:', train_labels.shape)
    print('Testing Features Shape:', test_features.shape)
    print('Testing Labels Shape:', test_labels.shape)

    # Instantiate model with 1000 decision trees
    rf = RandomForestClassifier(n_estimators = 1000)

    # Train the model on training data
    rf.fit(train_features, train_labels)

    print("Features Importances")
    print(rf.feature_importances_)

    predictions = rf.predict(test_features)


    report = classification_report(test_labels, predictions, np.unique(labels))
    return report

def get_metrics(report, tag, start_point):
    """
    Method that extract metrics from the string table returned by classification_report method from sklearn package
    """
    avg_precision = report[report.index(tag)+start_point:report.index(tag)+start_point+4]
    avg_recall = report[report.index(tag)+start_point+10:report.index(tag)+start_point+10+4]
    avg_f1_score = report[report.index(tag)+start_point+20:report.index(tag)+start_point+20+4]
    return avg_precision, avg_recall, avg_f1_score

def get_metrics_avg(number_iterations):
    """
    Method that make number_iterations training a RF classifier and return 2 dictionaires with metics
    
        - One with metrics by genre label
        - The other with total metrics of all the genre labels
    """
    
    # Initializing the by genre and total metrics arrays necessaries to catch the data
    
    blues_precision_array = np.arange(number_iterations).astype(np.float32)
    blues_recall_array = np.arange(number_iterations).astype(np.float32)
    blues_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    classical_precision_array = np.arange(number_iterations).astype(np.float32)
    classical_recall_array = np.arange(number_iterations).astype(np.float32)
    classical_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    country_precision_array = np.arange(number_iterations).astype(np.float32)
    country_recall_array = np.arange(number_iterations).astype(np.float32)
    country_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    disco_precision_array = np.arange(number_iterations).astype(np.float32)
    disco_recall_array = np.arange(number_iterations).astype(np.float32)
    disco_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    hiphop_precision_array = np.arange(number_iterations).astype(np.float32)
    hiphop_recall_array = np.arange(number_iterations).astype(np.float32)
    hiphop_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    jazz_precision_array = np.arange(number_iterations).astype(np.float32)
    jazz_recall_array = np.arange(number_iterations).astype(np.float32)
    jazz_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    metal_precision_array = np.arange(number_iterations).astype(np.float32)
    metal_recall_array = np.arange(number_iterations).astype(np.float32)
    metal_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    pop_precision_array = np.arange(number_iterations).astype(np.float32)
    pop_recall_array = np.arange(number_iterations).astype(np.float32)
    pop_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    reggae_precision_array = np.arange(number_iterations).astype(np.float32)
    reggae_recall_array = np.arange(number_iterations).astype(np.float32)
    reggae_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    rock_precision_array = np.arange(number_iterations).astype(np.float32)
    rock_recall_array = np.arange(number_iterations).astype(np.float32)
    rock_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    macro_avg_precision_array = np.arange(number_iterations).astype(np.float32)
    macro_avg_recall_array = np.arange(number_iterations).astype(np.float32)
    macro_avg_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    weighted_avg_precision_array = np.arange(number_iterations).astype(np.float32)
    weighted_avg_recall_array = np.arange(number_iterations).astype(np.float32)
    weighted_avg_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    accuracy_avg_array = np.arange(number_iterations).astype(np.float32)
    
    labels = ['Blues','Classical','Country','Disco','Hiphop','Jazz','Metal','Pop','Reggae','Rock']
    
    # Loop with the training RF classiffier itrerations. We fill the arrays metrics in each iteration     
    
    for i in range(0, number_iterations):
        report = train_clasify()
        
        # Metrics by Genre
        (precision, recall, f1_score) = get_metrics(report, "Blues", 12)
        blues_precision_array[i] = precision
        blues_recall_array[i] = recall
        blues_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Classical", 16)
        classical_precision_array[i] = precision
        classical_recall_array[i] = recall
        classical_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Country", 14)
        country_precision_array[i] = precision
        country_recall_array[i] = recall
        country_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Disco", 12)
        disco_precision_array[i] = precision
        disco_recall_array[i] = recall
        disco_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Hiphop", 13)
        hiphop_precision_array[i] = precision
        hiphop_recall_array[i] = recall
        hiphop_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Jazz", 11)
        jazz_precision_array[i] = precision
        jazz_recall_array[i] = recall
        jazz_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Metal", 12)
        metal_precision_array[i] = precision
        metal_recall_array[i] = recall
        metal_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Pop", 10)
        pop_precision_array[i] = precision
        pop_recall_array[i] = recall
        pop_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Reggae", 13)
        reggae_precision_array[i] = precision
        reggae_recall_array[i] = recall
        reggae_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Rock", 11)
        rock_precision_array[i] = precision
        rock_recall_array[i] = recall
        rock_f1_score_array[i] = f1_score
        
        # General metrics         
        
        (precision, recall, f1_score) = get_metrics(report, "accuracy", 15)
        accuracy_avg_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "weighted", 19)
        weighted_avg_precision_array[i] = precision
        weighted_avg_recall_array[i] = recall
        weighted_avg_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "macro avg", 16)
        macro_avg_precision_array[i] = precision
        macro_avg_recall_array[i] = recall
        macro_avg_f1_score_array[i] = f1_score
        
    # Now we fill the arrays that we are going to use to create the final dictionaries     
    
    precision_array = []
    recall_array = []
    f1_score_array = []
    
    precision_array.append(blues_precision_array.mean())
    recall_array.append(blues_recall_array.mean())
    f1_score_array.append(blues_f1_score_array.mean())
    
    precision_array.append(classical_precision_array.mean())
    recall_array.append(classical_recall_array.mean())
    f1_score_array.append(classical_f1_score_array.mean())
    
    precision_array.append(country_precision_array.mean())
    recall_array.append(country_recall_array.mean())
    f1_score_array.append(country_f1_score_array.mean())
    
    precision_array.append(disco_precision_array.mean())
    recall_array.append(disco_recall_array.mean())
    f1_score_array.append(disco_f1_score_array.mean())
    
    precision_array.append(hiphop_precision_array.mean())
    recall_array.append(hiphop_recall_array.mean())
    f1_score_array.append(hiphop_f1_score_array.mean())
    
    precision_array.append(jazz_precision_array.mean())
    recall_array.append(jazz_recall_array.mean())
    f1_score_array.append(jazz_f1_score_array.mean())
    
    precision_array.append(metal_precision_array.mean())
    recall_array.append(metal_recall_array.mean())
    f1_score_array.append(metal_f1_score_array.mean())
    
    precision_array.append(pop_precision_array.mean())
    recall_array.append(pop_recall_array.mean())
    f1_score_array.append(pop_f1_score_array.mean())
    
    precision_array.append(reggae_precision_array.mean())
    recall_array.append(reggae_recall_array.mean())
    f1_score_array.append(reggae_f1_score_array.mean())
    
    precision_array.append(rock_precision_array.mean())
    recall_array.append(rock_recall_array.mean())
    f1_score_array.append(rock_f1_score_array.mean())
    
    # Dictionaries creation     
    
    by_label_metrics_dict = {
        
        'Genre': labels,
        'avg_precision': precision_array,
        'avg_recall': recall_array,
        'avg_f1_score': f1_score_array,
        
    }
    
    total_metrics_dict = {
        
        'avg_accuracy': accuracy_avg_array.mean(),
        
        'weighted_avg_precision': weighted_avg_precision_array.mean(),
        'weighted_avg_recall': weighted_avg_recall_array.mean(),
        'weighted_avg_f1_score': weighted_avg_f1_score_array.mean(),
        
        'macro_avg_precision': macro_avg_precision_array.mean(),
        'macro_avg_recall': macro_avg_recall_array.mean(),
        'macro_avg_f1_score': macro_avg_f1_score_array.mean(),
        
    }

    return by_label_metrics_dict, total_metrics_dict


def export_genre_result_dict_to_csv(by_label_metrics_dict, total_metrics_dict):
    """
    Method that export two dictionaries to csv files using pandas dataframes
    
        - One with metrics by genre label
        - The other with total metrics of all the genre labels
    """
    by_label_metrics_dataframe_columns_tags = ['Genre', 'avg_precision', 'avg_recall', 'avg_f1_score']
    total_metrics_dataframe_columns_tags = ['avg_accuracy', 
                                            'weighted_avg_precision', 'weighted_avg_recall', 'weighted_avg_f1_score',
                                            'macro_avg_precision', 'macro_avg_recall', 'macro_avg_f1_score'
                                           ]

    df_by_label = pd.DataFrame(by_label_metrics_dict, columns = by_label_metrics_dataframe_columns_tags)
    df_total = pd.DataFrame(total_metrics_dict, columns = total_metrics_dataframe_columns_tags, index=[0])

    df_by_label.to_csv('by_label_metrics.csv')
    df_total.to_csv('total_metrics.csv')
    
    
(by_label_metrics_dict, total_metrics_dict) = get_metrics_avg(100)
export_genre_result_dict_to_csv(by_label_metrics_dict, total_metrics_dict)


Training Features Shape: (750, 35)
Training Labels Shape: (750,)
Testing Features Shape: (250, 35)
Testing Labels Shape: (250,)
Features Importances
[0.02664766 0.02943209 0.03437819 0.01651336 0.03994778 0.02268522
 0.01540977 0.03033022 0.04200852 0.0397666  0.04074001 0.03770849
 0.03743531 0.03407237 0.04256864 0.05043736 0.02375891 0.01520563
 0.02774925 0.02896173 0.03834564 0.03515464 0.02490953 0.02118584
 0.01697803 0.02167329 0.01949917 0.01805407 0.02289662 0.02042493
 0.02206371 0.02809872 0.0247546  0.02576437 0.02443974]
Training Features Shape: (750, 35)
Training Labels Shape: (750,)
Testing Features Shape: (250, 35)
Testing Labels Shape: (250,)


C:\Users\alvar\anaconda3\envs\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Features Importances
[0.03020391 0.0300553  0.03182561 0.01597546 0.03827857 0.0271517
 0.0165973  0.03009038 0.0400685  0.03349708 0.04234209 0.03575597
 0.03389374 0.03170019 0.04140507 0.04773033 0.02706564 0.01586071
 0.03084433 0.03039594 0.03994553 0.0358905  0.0268085  0.02405488
 0.01601719 0.02142369 0.01850128 0.01796945 0.02065416 0.0205304
 0.02056729 0.02425852 0.02536824 0.0311696  0.02610295]
Training Features Shape: (750, 35)
Training Labels Shape: (750,)
Testing Features Shape: (250, 35)
Testing Labels Shape: (250,)


C:\Users\alvar\anaconda3\envs\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Features Importances
[0.02736755 0.02835254 0.03179478 0.01555197 0.03772523 0.02775345
 0.01564647 0.02897196 0.04232971 0.03466831 0.04544577 0.03598672
 0.03554162 0.03076503 0.04300617 0.04805502 0.02700159 0.01619542
 0.02870758 0.02992032 0.03836086 0.03751202 0.02363524 0.02240195
 0.01628599 0.0214949  0.01823973 0.01729099 0.02173195 0.02096186
 0.0226825  0.02791116 0.02756785 0.02825014 0.02488567]
Training Features Shape: (750, 35)
Training Labels Shape: (750,)
Testing Features Shape: (250, 35)
Testing Labels Shape: (250,)


...

C:\Users\alvar\anaconda3\envs\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Features Importances
[0.0278063  0.02945346 0.03315984 0.01593958 0.03655023 0.02765584
 0.01400072 0.02995497 0.04314904 0.03541782 0.04037559 0.03402418
 0.03579133 0.03466447 0.04461888 0.04968486 0.02831043 0.01390026
 0.02822233 0.02809697 0.04230793 0.03634625 0.02556359 0.02170875
 0.01553458 0.02163743 0.01788278 0.0180943  0.02172192 0.0209978
 0.0217691  0.02635024 0.02725485 0.02801293 0.02404043]
Training Features Shape: (750, 35)
Training Labels Shape: (750,)
Testing Features Shape: (250, 35)
Testing Labels Shape: (250,)


C:\Users\alvar\anaconda3\envs\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Features Importances
[0.03048487 0.03108689 0.02860215 0.01631857 0.03380734 0.02828506
 0.01617205 0.0273322  0.04391472 0.03745756 0.04560507 0.03639296
 0.03678839 0.03065348 0.04362546 0.04859012 0.02754103 0.01638638
 0.02834787 0.02972584 0.03545756 0.03054468 0.02400955 0.02148679
 0.01599881 0.02262007 0.01686782 0.01776774 0.02133117 0.02120345
 0.02341079 0.03036525 0.02880114 0.02857832 0.02443883]


C:\Users\alvar\anaconda3\envs\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [ ]:
import junix;
junix.export_images(filepath= "ClassificationTree.ipynb")


In [6]:
plot_confusion_matrix(rf, test_features, test_labels)
fig = plt.gcf()
fig.set_size_inches(28.5, 28.5)
plt.show()

import junix;
junix.export_images(filepath= "ClassificationTree.ipynb")

NameError: name 'rf' is not defined